### Data Augmentation Techniques
* Scaling
* Translation
* Rotation (at 90 degrees)
* Rotation (at finer angles)
* Flipping
* Adding Salt and Pepper noise
* Lighting condition
* Perspective transform

In [ ]:
import os
import numpy as np

from PIL import Image, ImageEnhance, ImageOps, ImageFilter
import glob
import math

%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

## Change image size if needed


In [ ]:
IMAGE_SIZE = 32


#### Scaling:
Having differently scaled object of interest in the images is the most important aspect of image diversity. When your network is in hands of real users, the object in the image can be tiny or large. Also, sometimes, object can cover the entire image and yet will not be present totally in image (i.e cropped at edges of object). The code shows scaling of image centrally.

#### Translation:
We would like our network to recognize the object present in any part of the image. Also, the object can be present partially in the corner or edges of the image. For this reason, we shift the object to various parts of the image. This may also result in addition of a background noise. The code snippet shows translating the image at four sides retaining 80 percent of the base image.

#### Rotation (at 90 degrees):
The network has to recognize the object present in any orientation. Assuming the image is square, rotating the image at 90 degrees will not add any background noise in the image.

#### Rotation (at finer angles):
Depending upon the requirement, there maybe a necessity to orient the object at minute angles. However problem with this approach is, it will add background noise. If the background in image is of a fixed color (say white or black), the newly added background can blend with the image. However, if the newly added background color doesn’t blend, the network may consider it as to be a feature and learn unnecessary features.

#### Flipping:
This scenario is more important for network to remove biasness of assuming certain features of the object is available in only a particular side. Consider the case shown in image example. You don’t want network to learn that tilt of banana happens only in right side as observed in the base image. Also notice that flipping produces different set of images from rotation at multiple of 90 degrees.My additional question is has anyone done some study on what is the maximum number of classes it gives good performance. Consider, data can be generated with good amount of diversity for each class and time of training is not a factor.

#### Adding Salt and Pepper noise:
Salt and Pepper noise refers to addition of white and black dots in the image. Though this may seem unnecessary, it is important to remember that a general user who is taking image to feed into your network may not be a professional photographer. His camera can produce blurry images with lots of white and black dots. This augmentation aides the above mentioned users.


#### Lighting condition:
This is a very important type of diversity needed in the image dataset not only for the network to learn properly the object of interest but also to simulate the practical scenario of images being taken by the user. The lighting condition of the images are varied by adding Gaussian noise in the image.

#### Perspective transform:
In perspective transform, we try to project image from a different point of view. For this, the position of object should be known in advance. Merely calculating perspective transform without knowing the position of the object can lead to degradation of the dataset. Hence, this type of augmentation has to be performed selectively. The greatest advantage with this augmentation is that it can emphasize on parts of object in image which the network needs to learn.

## Generate and save augmented images

In [ ]:
def gaussian_noise(img, path ):   
    idx = 0
    # red, green, blue = img.split()
    for loc in range(0,5):
        for scale in range(30,60,5):
            for y in range(img.height):
                for x in range(img.width):
                    gauss = np.random.normal(loc,scale)
                    value = img.getpixel((x, y))
                    image_red = (value[0] + int(gauss), value[1], value[2])
                    img.putpixel((x, y), image_red)
            
            filename = "img_gaussian_noise_{0}".format(idx)
            new_path = path.format(filename)
            img.save(new_path)
            idx += 1

In [ ]:
def box_blur(img, path ):   
    idx = 0
    for radius in range(1,5):
            img = img.filter(ImageFilter.BoxBlur(radius))
            filename = "img_box blur_{0}".format(idx)
            new_path = path.format(filename)
            img.save(new_path)
            idx += 1

In [ ]:
def crop_center(pil_img, crop_width, crop_height):
    img_width, img_height = pil_img.size
    return pil_img.crop(((img_width - crop_width) // 2,
                         (img_height - crop_height) // 2,
                         (img_width + crop_width) // 2,
                         (img_height + crop_height) // 2))

In [ ]:
def ScaleRotateTranslate(image, angle, center = None, new_center = None, scale = None,expand=False):
    if center is None:
        return image.rotate(angle)
    angle = -angle/180.0*math.pi
    nx,ny = x,y = center
    sx=sy=1.0
    if new_center:
        (nx,ny) = new_center
    if scale:
        (sx,sy) = scale
    cosine = math.cos(angle)
    sine = math.sin(angle)
    a = cosine/sx
    b = sine/sx
    c = x-nx*a-ny*b
    d = -sine/sy
    e = cosine/sy
    f = y-nx*d-ny*e
    return image.transform(image.size, Image.AFFINE, (a,b,c,d,e,f), resample=Image.BICUBIC)

In [ ]:
def scaled_image(img,path):
    # scale = [0.97,0.96,0.95,0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.87,0.86,0.85,0.84,0.80,0.75,0.70,0.65,0.60]
    scale = [0.99,0.98,0.97,0.96,0.95]
    for i in range(0, len(scale)):
        new_length = int(IMAGE_SIZE/scale[i])
        scaled_size = (new_length,new_length)
        new_image = img.resize(scaled_size)
        new_image = crop_center(new_image, IMAGE_SIZE, IMAGE_SIZE)
        # new_image = new_image.crop(((new_length - IMAGE_SIZE)/2 ,(new_length - IMAGE_SIZE)/2, IMAGE_SIZE,IMAGE_SIZE )) #(left, top, right, bottom)
        new_image = new_image.resize((IMAGE_SIZE,IMAGE_SIZE))
        filename = "img_scale_{0}".format(i)
        new_path = path.format(filename)
        new_image.save(new_path)

In [ ]:
def widen_image(img,path):
    scale = [0.97,0.96,0.95,0.94,0.93,0.92,0.91,0.90,0.89,0.88,0.87,0.86,0.85,0.84,0.80,0.75,0.70,0.65,0.60]
    # scale = [0.99,0.98,0.97,0.96,0.95]
    for i in range(0, len(scale)):
        new_length = int(IMAGE_SIZE/scale[i])
        scaled_size = (new_length,IMAGE_SIZE)
        new_image = img.resize(scaled_size)
        new_image = crop_center(new_image, IMAGE_SIZE, IMAGE_SIZE)
        # new_image = new_image.crop(((new_length - IMAGE_SIZE)/2 ,(new_length - IMAGE_SIZE)/2, IMAGE_SIZE,IMAGE_SIZE )) #(left, top, right, bottom)
        new_image = new_image.resize((IMAGE_SIZE,IMAGE_SIZE))
        filename = "img_widen_{0}".format(i)
        new_path = path.format(filename)
        new_image.save(new_path)

In [ ]:
def contrast_image(img,path):
    factor = [0.90,0.80,0.70,0.60,0.50,0.40,0.30,0.20,0.10]
    for i in range(0, len(factor)):
        filter = ImageEnhance.Contrast(img)
        new_image = filter.enhance(factor[i])
        filename = "img_contrast_" + str(i)
        new_path = path.format(filename)
        new_image.save(new_path)
        

In [ ]:
def tranlate_image(folder, img):
    translated_imgs = translate_images(img)
    
    for i in range(-4, 4):
        filename = "img_translate_{0}".format(i)
        filepath = new_img_name(folder, filename)

In [ ]:
def shear(img,path):
    for x in range(-10,10):
        transform = (1,x/100,0,
                     0,1,0)
        new_image = image.transform(image.size, Image.AFFINE, transform, resample=Image.BICUBIC)
        filename = "img_shearx_" + str(x)
        new_path = path.format(filename)
        new_image.save(new_path)
    for y in range(-10,10):
        transform = (1,0,0,
                     y/100,1,0)
        new_image = image.transform(image.size, Image.AFFINE, transform, resample=Image.BICUBIC)
        filename = "img_sheary_" + str(y)
        new_path = path.format(filename)
        new_image.save(new_path)

In [ ]:
def rotate_general_image(img, path):
    for degree in range(-10,10):
        new_image = ScaleRotateTranslate(img,degree)
        filename = "img_rotate_{0}".format(degree)
        new_path = path.format(filename)
        new_image.save(new_path)

In [ ]:
def invert(img, path):
    # try:
    im_invert = ImageOps.invert(img)
    # except:
    #     return
    filename = "img_inverted_"
    new_path = path.format(filename)
    im_invert.save(new_path)
    # print(path, new_path)

In [ ]:
def exec_images(img, path):
    # Reduce contrast 97% - 60%
    contrast_image(img, path)
    # Scale 97% - 60%
    scaled_image(img, path)
    # Widen 97% - 60%
    widen_image(img, path)
    # Box blur 1 -5
    box_blur(img, path)
    # # Tranlate the images
    # tranlate_image(i, img)
    # # Rotate the image -10 to 10 degress
    rotate_general_image(img, path)
    # Shear the image
    shear(img, path)
    # # # Flip the image
    # # flipped_image(i, img)
    # # Add noise in pixels
    # salt_pepper(i, img)
    # Add gaussian noise in pixels
    gaussian_noise(img, path)
    # # Lighting condition
    # lighting(i, img)
    # print("Processing done!")

## Apply image augmentation

In [ ]:
target_folders = glob.glob("augmented/*/*.*")
img_paths = ['{}'.format(file) for file in target_folders]
# filename = 'img_0.png'
# path = ''

for img_path in img_paths:
        path = img_path.split(".")[-2] + "_{0}." + img_path.split(".")[-1] #  
        # print(path)
        image = Image.open(img_path).convert('RGB')
        newsize = (IMAGE_SIZE, IMAGE_SIZE)
        image = image.resize(newsize)
        plt.imshow(image)
        exec_images(image, path)
    

## Create inverted images

In [ ]:
dirList = glob.glob("augmented/*/*.*")
img_paths = ['{}'.format(file) for file in dirList]

for img_path in img_paths:
        path = img_path.split(".")[-2] + "_{0}." + img_path.split(".")[-1]
        image = Image.open(img_path)
        invert(image, path)

In [ ]:
dirList = glob.glob("augmented/*/*.*")
img_paths = ['{}'.format(file) for file in dirList]

for img_path in img_paths:
        path = img_path.split(".")[-2] + "_{0}." + img_path.split(".")[-1]
        artefact_image = Image.open(img_path).convert('RGB')
        JPG_PATH = img_path.split(".")
        JPG_PATH.pop()
        JPG_PATH = "/".join(JPG_PATH) + ".jpg"
        # print(PATH+JPG_PATH)
        artefact_image.save(JPG_PATH, quality=5) # we want compression artefacts